In [2601]:
import numpy as np
import math

t = 0  # number of this round
T = 150000  # numbers of rounds
l = 30  # ||a|| <= L
d = 3 #dimension of arm
m = 1 #||theta|| < m
theta_stimato = [0.0, 0.0, 0.0] #np.array([])  theta
A = [] #collector of arm
num_arms = 0 #number of arms
num_theta = 0 #num theta
Th = [] #collector of theta
W = [] #distribution of probability to play arms
N = [] #number of times an arm is played
Y = [] #collection of reward
H = [] #collection of hedge losses
L = [] #collection of losses
Mixability_Gap = [] #collection ofa all mixability gap at each t
AI = [] #number of the arm played at time I
nu = 0.01 #regularization paramether
eta = 0.1 #learning paramether
theta_vero = 0
curr_answ = 0

def normalize(v):
    sum = np.sum(v)
    if sum > 0:
        return v / sum
    else:
        return np.array([0.33]*len(v))
    
class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        #i = 0
        #best = -5
        #while i < num_arms:
            #tmp = np.dot( A[i].arm_vector, np.transpose(self.vector))
            #if best < tmp:
                #best = tmp
                #self.best_arm = i

class arm():
    arm_vector = np.array([])
    id = 0
    best_theta = -1

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A
        A.append(self)
        W.append(0)
        N.append(0)
        L.append(0)
        self.id = num_arms
        num_arms +=1
        max = -5
        for i in range (0, num_theta):
            tmp = np.matmul(self.arm_vector, Th[i].vector)
            if tmp > max:
                self.best_theta = i
                max = tmp
                            

    def pull_arm(self):
        y = np.dot(self.arm_vector, np.transpose(Th[theta_vero].vector)) + np.random.normal(0, 1)
        Y.append(y)
        N[self.id] += 1
        AI.append(self.id)
        return y
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

def design_matrix(w):
    V = []
    i = 0
    while i < num_arms:
        b = [A[i].arm_vector]
        a = np.swapaxes(b, 0, 1)
        ris = np.matmul(a, b) * w[i]
        if (i == 0):
            V = ris
        else:
            V += ris
        i+=1
    return V

def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def arm_to_play():
    diff = np.array(N) - np.array(W)
    i = 0
    arm = 0
    min = +5000
    while i < num_arms:
        if diff[i] < min:
            min = diff[i]
            arm = i
        i += 1
    return arm

def stima_theta():
    V = design_matrix(N)
    I = np.identity(d)
    p = np.linalg.inv(V + nu*I)
    sum = 0
    for i in range (0, t):
        sum += Y[i] * np.array(A[AI[i]].arm_vector)
    res = np.matmul(p, sum)
    global theta_stimato
    theta_stimato = res
    return res

def best_answer():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, theta_stimato)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best

def best_arm():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, Th[theta_vero].vector)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best


def beta(time, delta):
    n = np.abs(np.random.normal(0, 1))
    res = math.sqrt(math.log10(1/delta) + d/2*math.log10(1 + (time*(l**2))/(n*d))) + math.sqrt(n/2)*m
    return res

def best_response():
    min = 1000000
    theta_adv = -11
    for i in range (0, num_theta):
        if not i == A[curr_answ].best_theta:
            curr = (norm_2_mat(np.array(theta_stimato) - np.array(Th[i].vector), design_matrix(W)))**2
            if curr < min:
                min = curr
                theta_adv = i
    return theta_adv

def calcola_U (id_arm):
    xi = [0]*d
    q2 = 4*(l**2)*(m**2)
    q1 = 0
    p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response()].vector)), np.array(A[id_arm].arm_vector))
    p2 = math.sqrt(2*beta(T, 1/T**3))*norm_2_mat(np.array(A[id_arm].arm_vector), design_matrix(normalize(N)) + np.random.normal(0, 1)*np.identity(d))
    r1 = (p1 + p2)**2
    r2 = (p1 - p2)**2
    if r1> r2:
        q1 = r1
    else:
        q1 = r2
    if q1 > q2:
        return q2
    else:
        return q1

def calcola_loss():
    loss = 0
    arm_play = arm_to_play()
    best_res = best_response ()
    for i in range (0, num_arms):
        loss += W[i]*calcola_U(i)/2
    L[arm_play] += (4*(l**2)*(m**2) - loss)/1000
    return (4*(l**2)*(m**2) - loss)/1000

In [2602]:
len([0,1])

2

In [2603]:
s = theta([0.1, 0.4, 0.5])
p = theta([0.4, 0.6, 0.2])
q = theta([0.01, 0.1, 0.8])

In [2604]:
a = arm([1,2,3])
b = arm([2,5,1])
c = arm([1, 3, 1])
w = [0.34, 0.33, 0.33]
V = design_matrix(w)

In [2605]:
np.linalg.norm(c.arm_vector)

3.3166247903554

In [2606]:
for i in range (0, num_arms):
    print(A[i].best_theta)

2
1
1


In [2607]:
arm_to_play()

0

In [2608]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero].vector))

2.4
2.7
1.8000000000000003


In [2609]:
def h():
    return np.dot(W, L)

def mix_loss():
    sum = 0
    for i in range (0, num_arms):
        sum += W[i] * math.exp(-eta * L[i])
    return -1/eta * np.log(sum)

def mixability_gap():
    m_g = h() - mix_loss()
    Mixability_Gap.append(m_g)

def new_eta():
    if t>0:
        sum = 0
        for i in range (0, t):
            sum += Mixability_Gap[i]
        eta = math.log(d)/sum
        
def generate_w():
    sum=0
    for i in range (0, num_arms):
        W[i] =math.exp(-eta * L[i])/num_arms
        sum += math.exp(-eta * L[i])/num_arms
    for i in range (0, num_arms):
        W[i] = W[i]/sum
    return W

In [2610]:
a.pull_arm()

2.511656093649707

In [2611]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

2.53951745565004


3.359821521675436

In [2612]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

2.1881655522663803


1.744076970871157

In [2613]:
N

[1, 1, 1]

In [2614]:
normalize(N)

array([0.33333333, 0.33333333, 0.33333333])

In [2615]:
for i in range (3000):
    t += 1
    stima_theta()
    best_answer()
    curr = arm_to_play()
    generate_w()
    err = calcola_loss()
    A[curr].pull_arm()

In [2616]:
print(theta_stimato)
print(Th[theta_vero].vector)
print(best_answer())
print(best_arm())
print(W)

[0.16657465 0.3749664  0.49970731]
[0.1, 0.4, 0.5]
1
1
[0.06713483892723414, 0.8517987400926234, 0.08106642098014245]


[1.9423824444604474, 3.175981218016338]

In [1595]:
b.pull_arm()
c.pull_arm()
b.pull_arm()

4.0553451251167205

In [1559]:
Y

[2.359809218555277,
 4.131155314683859,
 2.904517191004988,
 24.914421812754583,
 3.7649844287528778]

In [1560]:
N

[1, 3, 1]

In [1561]:
W = [1.1, 2.6, 1.3]

In [1562]:
arm_to_play()

2

In [1563]:
t = 4
stima_theta()

array([ 2.77437357, -0.06446984, -0.09043354])

In [1564]:
for i in range (0, num_arms):
    print(best_response(i))

1
0
0


In [1565]:
best_response(0)

1

In [1566]:
np.array(theta_stimato) - np.array(Th[best_response(0)].vector)

array([ 0.6, -0.1,  0.3])

In [1567]:
np.array(A[0].arm_vector)

array([1, 2, 3])

In [1568]:
p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response(0)].vector)), np.array(A[0].arm_vector))
p1

1.2999999999999998

In [1569]:
math.sqrt(2*beta(t, 1/t**3))

3.8072546006004324

In [1570]:
np.sqrt(2*beta(t, 1/t**3))

3.9841738037605214

In [1571]:
k = 1000
time = k
delta = 1/k**3


In [1572]:
n = np.abs(np.random.normal(0, 1))
n

1.3485216370993358

In [1573]:
math.sqrt(math.log10(1/delta))

3.0

In [1574]:
d/2*math.log10(1 + (time*(l**2))/(n*d))

6.589557593201865

In [1575]:
math.sqrt(n/2)*m

8.211338615290858

In [1576]:
beta(k, 1/k**3)

7.45007683426267

In [1577]:
p2 = np.sqrt(2*beta(t, 1/t**3))*norm_2_mat(np.array(A[0].arm_vector), design_matrix(N) + np.random.normal(0, 1)*np.identity(d))

In [1578]:
calcola_U(0, 2)

269601.97156973823

In [1579]:
calcola_U(1, 2)

649542.4283968954

In [1580]:
calcola_U(2, 2)

32689597.500705175

In [1581]:
for i in range (0 , T//3 - t):
    a.pull_arm()
    b.pull_arm()
    c.pull_arm()
    t += 3

In [1582]:

stima_theta()

array([0.40123031, 0.59863247, 0.20280594])

In [1583]:
for i in range (0, num_arms):
    print(best_response(i))

1
0
0


In [1584]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero].vector))

2.2
3.6
24.0


In [1585]:
for i in range (0, num_arms):
    print(calcola_U(i, best_response(i))/100000000)

63.064612560315304
462.9371160274454
16389.492404270248
